In [ ]:
# Установка transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 31.5 MB/s 
     |████████████████████████████████| 6.6 MB 54.7 MB/s 
     |████████████████████████████████| 163 kB 73.4 MB/s 


In [ ]:
#Установка catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.2 MB/s 


In [ ]:
import pandas as pd
from itertools import groupby

In [ ]:
#import необходимых модулей

import pandas as pd
import yaml
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
from datetime import datetime, timedelta

from catboost import CatBoostClassifier, Pool

In [ ]:
#Считывание данных в DataFrame 
train= pd.read_csv('train_net.csv', sep=';', index_col=None)
test= pd.read_csv('test_net.csv', sep=';', index_col=None)


In [ ]:
train

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,team_id,game_id,period_1,period_2,period_3,period_4,Период_1_всего,Период_2_всего,Период_3_всего,Период_4_всего,TestSummAll
0,10884,4,4,4,4,1440,202,Cancellation of bonus for violation of labor d...,"Contextual advertising, Advertising in print m...","Managers qualification, Anti-bureaucratic days...","Size of a reserve stock of WIP, The best aesth...",7,14,28,9,Cancellation of bonus for violation of labor d...
1,10106,5,4,5,5,1536,211,"Development of productionвЂ™s optimisation, Bo...","Repeat inspection of batch products, Developme...","Optimizing the placement of equipment, Increas...",R&D enhancement of aesthetic and ergonomic qua...,21,21,12,8,"Development of productionвЂ™s optimisation, Bo..."
2,10438,6,5,5,5,1742,242,"Manufacturing staff qualification, To place pa...",Minimal fixed + % by the results of a period (...,Quality of the purchased materials Raw (carbon...,"Factoring, Quality of the purchased materials ...",62,42,58,30,"Manufacturing staff qualification, To place pa..."
3,10130,5,4,5,5,1272,184,"Fabrication staff salaty, Analysis of the impo...","Health, Analysis of the market structure for t...","Education of the dealersвЂ™ sales staff, Quant...","Search and selection of dealers, Contracts of ...",24,24,33,63,"Fabrication staff salaty, Analysis of the impo..."
4,10667,4,5,5,5,1382,193,"Pick of contractors and providers, Engineers q...","Media-advertising, The best physical propertie...","Conduct customer analysis on the market, Firef...","Bicycles Production line 20000 units, Cumulati...",54,49,152,19,"Pick of contractors and providers, Engineers q..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,10563,5,4,5,4,1274,184,Quality of the purchased materials Raw (carbon...,Quantity of work shifts,"Development of productionвЂ™s optimisation, De...","Size of a reserve stock of FP, Employment of o...",10,1,11,4,Quality of the purchased materials Raw (carbon...
676,10386,5,4,5,5,1526,210,Medium strategy of staff recruitment,Bonuses for innovation and rationalization of ...,Cancellation of bonus for violation of labor d...,Sending Engineers on training courses,1,14,24,1,Medium strategy of staff recruitment Bonuses f...
677,10675,5,4,5,4,1670,235,"Optimisation of the loading unitвЂ™s shape, Th...","Size of a reserve stock of MR, Size of a reser...","Size of a reserve stock of MR, The volume of o...",The volume of of transportation - Large compan...,4,14,20,9,"Optimisation of the loading unitвЂ™s shape, Th..."
678,10366,4,4,4,4,1808,248,Improvement of the documentation of entrance c...,Sending staff on short-term courses of creativ...,Conducting regular medical examinations and br...,"Dividend payout in cash, Transfer the rights t...",10,44,45,38,Improvement of the documentation of entrance c...


In [ ]:
test

,id,team_id,game_id,period_1,period_2,period_3,period_4,Период_1_всего,Период_2_всего,Период_3_всего,Период_4_всего,TestSummAll
0,10199,1912,262,"Setting the base selling price, Quantity of wo...","Additions to reserve capital, Volume discounts",Development and functioning of the companyвЂ™s...,"Hoverbikes Production line 2500 units, Setting...",2,2,3,2,"Setting the base selling price, Quantity of wo..."
1,10539,1458,204,Additions to reserve capital,"Pick of contractors and providers, Additional ...","[Company #1] Acquisition of the company, Metho...","Bicycles Production line 40000 units, Increase...",1,9,12,13,Additions to reserve capital Pick of contracto...
2,10174,1348,190,Not decision,Improvement of the technology of entrance cont...,Establishment of mid-term contract - Industria...,"Quality of the purchased materials Lubricants""...",0,28,21,16,Not decision Improvement of the technology of ...
3,10465,1760,244,Improvement of the technology of entrance cont...,R&D enhancement of assortment qualities Standa...,"To sell overdue accounts receivable, which do ...","Research on elasticity of market demand, [long...",15,18,29,17,Improvement of the technology of entrance cont...
4,10066,1260,183,"Quantity of work shifts, Keep up the aesthetic...",Bonuses for the fulfillment of the production ...,"Development of productionвЂ™s optimisation, Qu...","Enguneers salary, Stepwise staff numerosityвЂ™...",7,23,16,6,"Quantity of work shifts, Keep up the aesthetic..."
...,...,...,...,...,...,...,...,...,...,...,...,...
288,10433,1562,214,"Pick of contractors and providers, Value of pr...",Analysis of the importance of product properti...,Quantity of work shifts,Not decision,15,6,1,0,"Pick of contractors and providers, Value of pr..."
289,10893,1278,184,R&D enhancement of assortment qualities Standa...,"Discipline, Moral qualities, Independently hir...",Collection information and statistics of the d...,"Creativity, Managers salary, Discipline, Hirin...",14,52,34,18,R&D enhancement of assortment qualities Standa...
290,10909,1474,206,Not decision,Not decision,Not decision,The volume of of transportation - Worldwide co...,0,0,0,1,Not decision Not decision Not decision The vol...
291,10889,1268,184,Renewal of checkpoints in production and stora...,"Security alarm upgrade, Evaluation of the char...","Advertising in print media, Radio advertising,...",Analysis of the product satisfaction,6,2,5,1,Renewal of checkpoints in production and stora...


In [ ]:
train.columns

Index(['id', 'Analytical thinking', 'Systemic thinking', 'Adaptability',
       'Focus', 'team_id', 'game_id', 'period_1', 'period_2', 'period_3',
       'period_4', 'Период_1_всего', 'Период_2_всего', 'Период_3_всего',
       'Период_4_всего', 'TestSummAll'],
      dtype='object')

In [ ]:
# выбираем поля для нейросети СatBoost по бинарной классификации для 1-й оценки


y_train=train[['Analytical thinking']]
X_train=train[['team_id', 'game_id', 'period_1', 'period_2', 'period_3',
       'period_4','Период_1_всего', 'Период_2_всего', 'Период_3_всего','Период_4_всего']]

testnet=test[['team_id', 'game_id', 'period_1', 'period_2', 'period_3',
       'period_4','Период_1_всего', 'Период_2_всего', 'Период_3_всего','Период_4_всего']]

In [ ]:
# Определяем веса для классов кластеров
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train), y = np.ravel(y_train))
class_weights = dict(zip(classes, weights))
class_weights

{1: 34.0,
 3: 3.7777777777777777,
 4: 0.3261390887290168,
 5: 0.6507177033492823,
 6: 9.714285714285714}

In [ ]:
#Обучаем 
pool_train_solution = Pool(X_train, y_train, cat_features = ['team_id', 'game_id'], text_features = ['period_1', 'period_2', 'period_3','period_4'])
pool_test_solution = Pool(testnet,cat_features = ['team_id', 'game_id'], text_features = ['period_1', 'period_2', 'period_3','period_4'])

model_test_1 = CatBoostClassifier(iterations=50,  task_type='GPU',depth=12, verbose =True, random_seed=24,# learning_rate=0.015,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'Poisson',objective = 'MultiClass',eval_metric = 'Accuracy',loss_function='Recall', class_weights=class_weights)
model_test_1.fit(pool_train_solution)

0:	learn: 0.8019559	total: 304ms	remaining: 14.9s
1:	learn: 0.8468695	total: 471ms	remaining: 11.3s
2:	learn: 0.8564458	total: 632ms	remaining: 9.9s
3:	learn: 0.8567163	total: 774ms	remaining: 8.9s
4:	learn: 0.8648629	total: 932ms	remaining: 8.39s
5:	learn: 0.8725253	total: 1.07s	remaining: 7.87s
6:	learn: 0.8830700	total: 1.26s	remaining: 7.75s
7:	learn: 0.8773192	total: 1.41s	remaining: 7.39s
8:	learn: 0.8840224	total: 1.56s	remaining: 7.1s
9:	learn: 0.8916801	total: 1.72s	remaining: 6.88s
10:	learn: 0.8974424	total: 1.87s	remaining: 6.63s
11:	learn: 0.9007997	total: 2.03s	remaining: 6.43s
12:	learn: 0.9065460	total: 2.18s	remaining: 6.21s
13:	learn: 0.9075052	total: 2.35s	remaining: 6.05s
14:	learn: 0.9065437	total: 2.51s	remaining: 5.86s
15:	learn: 0.9137379	total: 2.68s	remaining: 5.7s
16:	learn: 0.9170906	total: 2.84s	remaining: 5.51s
17:	learn: 0.9214026	total: 3.02s	remaining: 5.37s
18:	learn: 0.9223664	total: 3.18s	remaining: 5.19s
19:	learn: 0.9266807	total: 3.33s	remaining: 

In [ ]:
# предварительный тест схождения на всем обучающем множестве, добавление в трейн поля прогноза
y_pred = model_test_1.predict(X_train)
train['Analytical thinking_prognoz']=y_pred
print("Значение метрики recall на трейне : ", recall_score(y_train, y_pred , average='macro', zero_division=True))

Значение метрики recall на трейне :  0.8016244010942989


In [ ]:
#Получение ответов
y_pred_solution = model_test_1.predict(pool_test_solution)

In [ ]:
# Сравнение распределения теста и трейна
train['Analytical thinking'].value_counts()

4    417
5    209
3     36
6     14
1      4
Name: Analytical thinking, dtype: int64

In [ ]:
train['Analytical thinking_prognoz'].value_counts()

4    305
5    230
3    127
6     14
1      4
Name: Analytical thinking_prognoz, dtype: int64

In [ ]:
# Заводим результаты в тест и сравниваем
test['Analytical thinking']=y_pred_solution
test['Analytical thinking'].value_counts()

4    132
5    112
3     46
6      3
Name: Analytical thinking, dtype: int64

In [ ]:
# выбираем поле для целевой переменной 2-й оценки
y_train=train[['Systemic thinking']]

X_train=train[['team_id', 'game_id', 'period_1', 'period_2', 'period_3',
       'period_4',  'TestSummAll']]
testnet=test[['team_id', 'game_id', 'period_1', 'period_2', 'period_3',
       'period_4', 'TestSummAll']]

In [ ]:
# Определяем веса для классов кластеров
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train), y = np.ravel(y_train))
class_weights = dict(zip(classes, weights))
class_weights

{1: 34.0,
 3: 2.6666666666666665,
 4: 0.2769857433808554,
 5: 1.096774193548387,
 6: 13.6}

In [ ]:
#Обучаем 

pool_train_solution = Pool(X_train, y_train, cat_features = ['team_id', 'game_id'], text_features = ['period_1', 'period_2', 'period_3','period_4', 'TestSummAll'])
pool_test_solution = Pool(testnet,cat_features = ['team_id', 'game_id'], text_features = ['period_1', 'period_2', 'period_3','period_4', 'TestSummAll'])

model_test_2 = CatBoostClassifier(iterations=40,  task_type='GPU',depth=12, verbose =True, random_seed=24,# learning_rate=0.015,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'Poisson',objective = 'MultiClass',eval_metric = 'Accuracy',loss_function='Recall', class_weights=class_weights)
model_test_2.fit(pool_train_solution)

0:	learn: 0.8273674	total: 302ms	remaining: 11.8s
1:	learn: 0.8804362	total: 522ms	remaining: 9.91s
2:	learn: 0.8813876	total: 692ms	remaining: 8.54s
3:	learn: 0.8884147	total: 885ms	remaining: 7.96s
4:	learn: 0.9150483	total: 1.1s	remaining: 7.7s
5:	learn: 0.9215492	total: 1.27s	remaining: 7.21s
6:	learn: 0.9231785	total: 1.47s	remaining: 6.92s
7:	learn: 0.9357729	total: 1.68s	remaining: 6.74s
8:	learn: 0.9414592	total: 1.88s	remaining: 6.47s
9:	learn: 0.9504040	total: 2.05s	remaining: 6.16s
10:	learn: 0.9451416	total: 2.2s	remaining: 5.81s
11:	learn: 0.9496058	total: 2.38s	remaining: 5.57s
12:	learn: 0.9483838	total: 2.56s	remaining: 5.32s
13:	learn: 0.9528480	total: 2.75s	remaining: 5.11s
14:	learn: 0.9536791	total: 2.93s	remaining: 4.89s
15:	learn: 0.9557322	total: 3.14s	remaining: 4.71s
16:	learn: 0.9553249	total: 3.32s	remaining: 4.5s
17:	learn: 0.9532882	total: 3.5s	remaining: 4.27s
18:	learn: 0.9581762	total: 3.68s	remaining: 4.07s
19:	learn: 0.9601964	total: 3.86s	remaining: 3

In [ ]:
# предварительный тест схождения на всем обучающем множестве, добавление в трейн поля прогноза
y_pred = model_test_2.predict(X_train)
train['Systemic thinking_prognoz']=y_pred
print("Значение метрики recall на трейне : ", recall_score(y_train, y_pred , average='macro', zero_division=True))

Значение метрики recall на трейне :  0.7903940505313221


In [ ]:
#Получение ответов
y_pred_solution = model_test_2.predict(pool_test_solution)

In [ ]:
# Сравнение распределения теста и трейна
train['Systemic thinking'].value_counts()

4    491
5    124
3     51
6     10
1      4
Name: Systemic thinking, dtype: int64

In [ ]:
train['Systemic thinking_prognoz'].value_counts()

4    399
5    154
3     97
1     18
6     12
Name: Systemic thinking_prognoz, dtype: int64

In [ ]:
# Заводим результаты в тест и сравниваем
test['Systemic thinking']=y_pred_solution
test['Systemic thinking'].value_counts()

4    140
5     93
3     54
1      4
6      2
Name: Systemic thinking, dtype: int64

In [ ]:
# выбираем поле для целевой переменной 3-й оценки
y_train=train[['Adaptability']]

X_train=train[['team_id', 'game_id', 'period_1', 'period_2', 'period_3',
       'period_4']]
testnet=test[['team_id', 'game_id', 'period_1', 'period_2', 'period_3',
       'period_4']]

In [ ]:
# Определяем веса для классов кластеров
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train), y = np.ravel(y_train))
class_weights = dict(zip(classes, weights))
class_weights

{1: 56.666666666666664,
 2: 113.33333333333333,
 3: 3.1481481481481484,
 4: 0.36796536796536794,
 5: 0.35639412997903563,
 6: 7.555555555555555}

In [ ]:
#Обучаем 
pool_train_solution = Pool(X_train, y_train, cat_features = ['team_id', 'game_id'], text_features = ['period_1', 'period_2', 'period_3','period_4'])
pool_test_solution = Pool(testnet,cat_features = ['team_id', 'game_id'], text_features = ['period_1', 'period_2', 'period_3','period_4'])

model_test_3 = CatBoostClassifier(iterations=78,  task_type='GPU',depth=12, verbose =True, random_seed=24,# learning_rate=0.015,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'Poisson',objective = 'MultiClass',eval_metric = 'Accuracy',loss_function='Recall', class_weights=class_weights)
model_test_3.fit(pool_train_solution)

0:	learn: 0.8889808	total: 192ms	remaining: 14.8s
1:	learn: 0.8568105	total: 372ms	remaining: 14.1s
2:	learn: 0.8708911	total: 565ms	remaining: 14.1s
3:	learn: 0.8877284	total: 758ms	remaining: 14s
4:	learn: 0.8930205	total: 963ms	remaining: 14.1s
5:	learn: 0.9036388	total: 1.16s	remaining: 14s
6:	learn: 0.9105373	total: 1.34s	remaining: 13.6s
7:	learn: 0.9110784	total: 1.51s	remaining: 13.2s
8:	learn: 0.9132600	total: 1.69s	remaining: 13s
9:	learn: 0.9153394	total: 1.86s	remaining: 12.7s
10:	learn: 0.9175549	total: 2.04s	remaining: 12.5s
11:	learn: 0.9207336	total: 2.24s	remaining: 12.3s
12:	learn: 0.9229322	total: 2.4s	remaining: 12s
13:	learn: 0.9186372	total: 2.57s	remaining: 11.7s
14:	learn: 0.9239123	total: 2.73s	remaining: 11.5s
15:	learn: 0.9281732	total: 2.91s	remaining: 11.3s
16:	learn: 0.9244364	total: 3.08s	remaining: 11.1s
17:	learn: 0.9254846	total: 3.26s	remaining: 10.9s
18:	learn: 0.9276151	total: 3.44s	remaining: 10.7s
19:	learn: 0.9302356	total: 3.63s	remaining: 10.5s

In [ ]:
# предварительный тест схождения на всем обучающем множестве, добавление в трейн поля прогноза
y_pred = model_test_3.predict(X_train)
train['Adaptability_prognoz']=y_pred
print("Значение метрики recall на трейне : ", recall_score(y_train, y_pred , average='macro', zero_division=True))

Значение метрики recall на трейне :  0.7857784947407588


In [ ]:
#Получение ответов
y_pred_solution = model_test_3.predict(pool_test_solution)

In [ ]:
# Сравнение распределения теста и трейна
train['Adaptability'].value_counts()

5    318
4    308
3     36
6     15
1      2
2      1
Name: Adaptability, dtype: int64

In [ ]:
train['Adaptability_prognoz'].value_counts()

5    278
4    248
3    131
6     17
1      5
2      1
Name: Adaptability_prognoz, dtype: int64

In [ ]:
# Заводим результаты в тест и сравниваем
test['Adaptability']=y_pred_solution
test['Adaptability'].value_counts()

5    125
4    108
3     56
6      4
Name: Adaptability, dtype: int64

In [ ]:
# выбираем поле для целевой переменной 4-й оценки
y_train=train[['Focus']]

X_train=train[['team_id', 'game_id', 'period_1', 'period_2', 'period_3','Период_1_всего', 'Период_2_всего', 'Период_3_всего','Период_4_всего',
       'period_4',  'TestSummAll']]
testnet=test[['team_id', 'game_id', 'period_1', 'period_2', 'period_3','Период_1_всего', 'Период_2_всего', 'Период_3_всего','Период_4_всего',
       'period_4', 'TestSummAll']]


In [ ]:
# Определяем веса для классов кластеров
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train), y = np.ravel(y_train))
class_weights = dict(zip(classes, weights))
class_weights

{1: 37.77777777777778,
 2: 113.33333333333333,
 3: 3.9080459770114944,
 4: 0.2833333333333333,
 5: 0.4644808743169399,
 6: 37.77777777777778}

In [ ]:
#Обучаем 


pool_train_solution = Pool(X_train, y_train, cat_features = ['team_id', 'game_id'], text_features = ['period_1', 'period_2', 'period_3','period_4', 'TestSummAll'])
pool_test_solution = Pool(testnet,cat_features = ['team_id', 'game_id'], text_features = ['period_1', 'period_2', 'period_3','period_4', 'TestSummAll'])

model_test_4 = CatBoostClassifier(iterations=50,  task_type='GPU',depth=12, verbose =True, random_seed=24,# learning_rate=0.015,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'Poisson',objective = 'MultiClass',eval_metric = 'Accuracy',loss_function='Recall', class_weights=class_weights)
model_test_4.fit(pool_train_solution)

0:	learn: 0.8404122	total: 473ms	remaining: 23.2s
1:	learn: 0.8588951	total: 780ms	remaining: 18.7s
2:	learn: 0.8961859	total: 1.1s	remaining: 17.2s
3:	learn: 0.9162637	total: 1.44s	remaining: 16.6s
4:	learn: 0.9236612	total: 1.75s	remaining: 15.7s
5:	learn: 0.9342760	total: 2.06s	remaining: 15.1s
6:	learn: 0.9321926	total: 2.37s	remaining: 14.5s
7:	learn: 0.9374590	total: 2.68s	remaining: 14.1s
8:	learn: 0.9366598	total: 2.98s	remaining: 13.6s
9:	learn: 0.9410929	total: 3.3s	remaining: 13.2s
10:	learn: 0.9442418	total: 3.62s	remaining: 12.8s
11:	learn: 0.9439754	total: 3.92s	remaining: 12.4s
12:	learn: 0.9487090	total: 4.25s	remaining: 12.1s
13:	learn: 0.9513251	total: 4.56s	remaining: 11.7s
14:	learn: 0.9535246	total: 4.89s	remaining: 11.4s
15:	learn: 0.9543579	total: 5.21s	remaining: 11.1s
16:	learn: 0.9551913	total: 5.56s	remaining: 10.8s
17:	learn: 0.9562910	total: 5.88s	remaining: 10.5s
18:	learn: 0.9564413	total: 6.18s	remaining: 10.1s
19:	learn: 0.9579577	total: 6.53s	remaining

In [ ]:
# предварительный тест схождения на всем обучающем множестве, добавление в трейн поля прогноза
y_pred = model_test_4.predict(X_train)
train['Focus_prognoz']=y_pred
print("Значение метрики recall на трейне : ", recall_score(y_train, y_pred , average='macro', zero_division=True))

Значение метрики recall на трейне :  0.8161056937378305


In [ ]:
#Получение ответов
y_pred_solution = model_test_4.predict(pool_test_solution)

In [ ]:
train['Focus'].value_counts()

4    400
5    244
3     29
6      3
1      3
2      1
Name: Focus, dtype: int64

In [ ]:
# Сравнение распределения теста и трейна
train['Focus_prognoz'].value_counts()

4    390
5    247
3     37
6      3
1      2
2      1
Name: Focus_prognoz, dtype: int64

In [ ]:
# Заводим результаты в тест
test['Focus']=y_pred_solution
test['Focus'].value_counts()

4    178
5    102
3     13
Name: Focus, dtype: int64

In [ ]:
test_itog=test[['id','Analytical thinking', 'Systemic thinking', 'Adaptability', 'Focus']]
test_itog

,id,Analytical thinking,Systemic thinking,Adaptability,Focus
0,10199,4,4,4,4
1,10539,4,4,5,5
2,10174,5,5,4,4
3,10465,4,5,5,5
4,10066,5,4,4,4
...,...,...,...,...,...
288,10433,4,5,4,4
289,10893,5,5,5,5
290,10909,3,4,3,4
291,10889,3,3,3,4


In [ ]:
test_itog.to_csv('прогноз общий предварительный diff out chislo.csv', sep=',', index=None)

In [ ]:
# смотрим трейн с полями прогноза модели
train

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,team_id,game_id,period_1,period_2,period_3,period_4,Период_1_всего,Период_2_всего,Период_3_всего,Период_4_всего,TestSummAll,Analytical thinking_prognoz,Systemic thinking_prognoz,Adaptability_prognoz,Focus_prognoz
0,10884,4,4,4,4,1440,202,Cancellation of bonus for violation of labor d...,"Contextual advertising, Advertising in print m...","Managers qualification, Anti-bureaucratic days...","Size of a reserve stock of WIP, The best aesth...",7,14,28,9,Cancellation of bonus for violation of labor d...,4,3,4,4
1,10106,5,4,5,5,1536,211,"Development of productionвЂ™s optimisation, Bo...","Repeat inspection of batch products, Developme...","Optimizing the placement of equipment, Increas...",R&D enhancement of aesthetic and ergonomic qua...,21,21,12,8,"Development of productionвЂ™s optimisation, Bo...",3,5,4,5
2,10438,6,5,5,5,1742,242,"Manufacturing staff qualification, To place pa...",Minimal fixed + % by the results of a period (...,Quality of the purchased materials Raw (carbon...,"Factoring, Quality of the purchased materials ...",62,42,58,30,"Manufacturing staff qualification, To place pa...",5,5,5,5
3,10130,5,4,5,5,1272,184,"Fabrication staff salaty, Analysis of the impo...","Health, Analysis of the market structure for t...","Education of the dealersвЂ™ sales staff, Quant...","Search and selection of dealers, Contracts of ...",24,24,33,63,"Fabrication staff salaty, Analysis of the impo...",5,4,5,5
4,10667,4,5,5,5,1382,193,"Pick of contractors and providers, Engineers q...","Media-advertising, The best physical propertie...","Conduct customer analysis on the market, Firef...","Bicycles Production line 20000 units, Cumulati...",54,49,152,19,"Pick of contractors and providers, Engineers q...",5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,10563,5,4,5,4,1274,184,Quality of the purchased materials Raw (carbon...,Quantity of work shifts,"Development of productionвЂ™s optimisation, De...","Size of a reserve stock of FP, Employment of o...",10,1,11,4,Quality of the purchased materials Raw (carbon...,4,4,5,4
676,10386,5,4,5,5,1526,210,Medium strategy of staff recruitment,Bonuses for innovation and rationalization of ...,Cancellation of bonus for violation of labor d...,Sending Engineers on training courses,1,14,24,1,Medium strategy of staff recruitment Bonuses f...,5,4,5,5
677,10675,5,4,5,4,1670,235,"Optimisation of the loading unitвЂ™s shape, Th...","Size of a reserve stock of MR, Size of a reser...","Size of a reserve stock of MR, The volume of o...",The volume of of transportation - Large compan...,4,14,20,9,"Optimisation of the loading unitвЂ™s shape, Th...",4,4,4,4
678,10366,4,4,4,4,1808,248,Improvement of the documentation of entrance c...,Sending staff on short-term courses of creativ...,Conducting regular medical examinations and br...,"Dividend payout in cash, Transfer the rights t...",10,44,45,38,Improvement of the documentation of entrance c...,5,4,1,4


In [ ]:
# сохраняем для коррекции прогноза по командным играм
train.to_csv('train_to_team_games.csv', sep=',', index=None)